In [7]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import shutil
from shutil import copy
from collections import defaultdict
import tensorflow as tf
import keras.backend as K
from patchify import patchify, unpatchify
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
def delete_directory(path):
    try:
        shutil.rmtree(path)
        print(f"Directory {path} and its contents deleted successfully!")
    except FileNotFoundError:
        print(f"Directory {path} not found.")
    except PermissionError:
        print(f"Permission denied to delete {path}.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Images_for_model = r"C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\Images for model"

# delete_directory(Images_for_model)

# splited_masks = r"C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\splited_masks"

# delete_directory(splited_masks)

# Images_for_model2 = r"C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\images for model 2"

# delete_directory(Images_for_model2)

In [9]:
# Function to categorize the files and move them into folders
def categorize_and_copy_files(file_paths):
    categories = defaultdict(list)

    for file_path in file_paths:
        # Extracting the filename from the path
        file_name = os.path.basename(file_path)

        # Checking for keywords to categorize files
        if "root_mask" in file_name and "occluded" not in file_name:
            categories['root'].append(file_path)
        elif "shoot_mask" in file_name:
            categories['shoot'].append(file_path)
        elif "seed_mask" in file_name:
            categories['seed'].append(file_path)
        elif "occluded_root_mask" in file_name:
            categories['occluded_root'].append(file_path)

    # Create a parent directory for all categories
    parent_folder = os.path.join(os.getcwd(), "splited_masks")
    if not os.path.exists(parent_folder):
        os.makedirs(parent_folder)

    # Create folders for each category and copy files into them
    for category in categories:
        folder_path = os.path.join(parent_folder, category)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Copy files to respective folders
        for file in categories[category]:
            copy(file, os.path.join(folder_path, os.path.basename(file)))

In [10]:
folder_path = r"C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\Masks"

# List all file paths in the folder
file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]

# Categorize the files and copy them into folders
categorize_and_copy_files(file_paths)

In [11]:
import os
import shutil
from sklearn.model_selection import train_test_split

def restructure_images_and_masks(images_path, masks_path, dest_directory, mask_type='root'):
    # Creating necessary directories
    train_images_dir = os.path.join(dest_directory, f'{mask_type}', 'train_images', 'train')
    train_masks_dir = os.path.join(dest_directory, f'{mask_type}', 'train_masks', 'train')
    val_images_dir = os.path.join(dest_directory, f'{mask_type}', 'val_images', 'val')
    val_masks_dir = os.path.join(dest_directory, f'{mask_type}', 'val_masks', 'val')

    for directory in [train_images_dir, train_masks_dir, val_images_dir, val_masks_dir]:
        os.makedirs(directory, exist_ok=True)

    # Get list of image files
    image_files = os.listdir(images_path)

    # Splitting images into train and validation sets
    train_images, val_images = train_test_split(image_files, test_size=0.2, random_state=42)

    # Move images to respective directories
    def move_images(images, source_path, dest_image_path, dest_mask_path):
        for img in images:
            img_path = os.path.join(source_path, img)
            shutil.copy(img_path, os.path.join(dest_image_path, img))
            mask_name = img.replace('.png', f'_{mask_type}_mask.tif')  # Adjust mask file name
            mask_path = os.path.join(masks_path, mask_name)
            shutil.copy(mask_path, os.path.join(dest_mask_path, mask_name))

    move_images(train_images, images_path, train_images_dir, train_masks_dir)
    move_images(val_images, images_path, val_images_dir, val_masks_dir)
    

# Usage example:
images_path = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\Images\train'
masks_path = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\splited_masks\root'
dest_directory = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\images for model'

restructure_images_and_masks(images_path, masks_path, dest_directory, mask_type='root')

masks_path = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\splited_masks\seed'

restructure_images_and_masks(images_path, masks_path, dest_directory, mask_type='seed')

masks_path = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\splited_masks\shoot'

restructure_images_and_masks(images_path, masks_path, dest_directory, mask_type='shoot')

masks_path = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\splited_masks\occluded_root'

restructure_images_and_masks(images_path, masks_path, dest_directory, mask_type='occluded_root')


In [14]:
import os
import shutil

def create_test_and_mask_folders(test_images_path, mask_images_path, mask_type='root'):
    # Create directories for test images and masks
    test_images_dir = f'images for model/{mask_type}/test_images/test'
    test_masks_dir = f'images for model/{mask_type}/test_masks/test'

    os.makedirs(test_images_dir, exist_ok=True)
    os.makedirs(test_masks_dir, exist_ok=True)

    # List all test image files
    test_image_files = os.listdir(test_images_path)

    # Iterate through test images, find corresponding masks, and copy them to respective folders
    for image_file in test_image_files:
        # Match mask file name based on the image file name
        mask_file = image_file.replace('.png', f'_{mask_type}_mask.tif')
        mask_file_path = os.path.join(mask_images_path, mask_file)

        # Check if corresponding mask file exists
        if os.path.exists(mask_file_path):
            # Copy test image to test images directory
            shutil.copy(os.path.join(test_images_path, image_file), os.path.join(test_images_dir, image_file))

            # Copy corresponding mask image to test masks directory
            shutil.copy(mask_file_path, os.path.join(test_masks_dir, mask_file))
        else:
            print(f"No mask found for {image_file}.")




In [15]:
test_images_path = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\Images\test'
mask_images_path = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\splited_masks\root'

create_test_and_mask_folders(test_images_path, mask_images_path,mask_type='root')


mask_images_path = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\splited_masks\seed'

create_test_and_mask_folders(test_images_path, mask_images_path,mask_type='seed')


mask_images_path = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\splited_masks\shoot'

create_test_and_mask_folders(test_images_path, mask_images_path,mask_type='shoot')


mask_images_path = r'C:\Users\domin\Desktop\Year 2 Block B\2023-24b-fai2-adsai-DominikSzewczyk224180\1. Computer Vision\Task4\splited_masks\occluded_root'

create_test_and_mask_folders(test_images_path, mask_images_path,mask_type='occluded_root')



